In [4]:
import pandas as pd
import os
os.chdir('/Users/vodnalakarthik/Desktop')


# -------------------------------
# STEP 1: Load All Required CSVs
# -------------------------------
df_customers = pd.read_csv('cleaned_customers.csv')               # Main customer data
df_gender = pd.read_csv('gender_distribution.csv')               # Gender distribution (summary)
df_revenue = pd.read_csv('revenue_by_category.csv')              # Revenue per category
df_age_groups = pd.read_csv('customer_age_groups.csv')           # Age group totals
df_avg_spending = pd.read_csv('avg_spending.csv')                # Peer group avg income & spending
df_high_last_amount = pd.read_csv('high_last_amount.csv')        # Customers with high last purchase

# -------------------------------
# STEP 2: Add Age_Group to df_customers
# -------------------------------
def assign_age_group(age):
    if age < 25:
        return 'Under 25'
    elif 25 <= age <= 35:
        return '25–35'
    elif 36 <= age <= 50:
        return '36–50'
    else:
        return '51+'

df_customers['Age_Group'] = df_customers['Age'].apply(assign_age_group)

# -------------------------------
# STEP 3: Join Age Group Totals to df_customers
# -------------------------------
df_customers = pd.merge(df_customers, df_age_groups, on='Age_Group', how='left')

# -------------------------------
# STEP 4: Merge With Peer Group Averages (Table 4)
# -------------------------------
# avg_spending.csv must have: Category, Gender, Average_Income, Average_Spending_Score
df_seg = pd.merge(df_customers, df_avg_spending, on=['Category', 'Gender'], how='left')

# -------------------------------
# STEP 5: Calculate Differences vs Peer Group
# -------------------------------
df_seg['Spend_vs_Peer'] = df_seg['Spending_Score'] - df_seg['Average_Spending_Score']
df_seg['Income_vs_Peer'] = df_seg['Income'] - df_seg['Average_Income']

# -------------------------------
# STEP 6: Tag Each Customer Based on Performance
# -------------------------------
def tag_peer_segment(row):
    if row['Spend_vs_Peer'] > 0 and row['Income_vs_Peer'] > 0:
        return 'Top Performer in Peer Group'
    elif row['Spend_vs_Peer'] < 0 and row['Income_vs_Peer'] > 0:
        return 'Under-Spending High Earner'
    elif row['Spend_vs_Peer'] > 0 and row['Income_vs_Peer'] < 0:
        return 'Over-Spending Low Earner'
    else:
        return 'Average or Below'

df_seg['Peer_Segment'] = df_seg.apply(tag_peer_segment, axis=1)

# -------------------------------
# STEP 7: Clean Unnecessary Columns
# -------------------------------
if 'ID_x' in df_seg.columns:
    df_seg.rename(columns={'ID_x': 'ID'}, inplace=True)
df_seg.drop(columns=['ID_y'], inplace=True, errors='ignore')

# -------------------------------
# STEP 8: Select Final Columns for Export
# -------------------------------
final_cols = [
    'ID', 'Age', 'Age_Group', 'Gender', 'Category', 'Income',
    'Spending_Score', 'Average_Income', 'Average_Spending_Score',
    'Spend_vs_Peer', 'Income_vs_Peer', 'Peer_Segment', 'Total_Customers'
]

print("✅ Final Table Preview:")
print(df_seg[final_cols].head())

# -------------------------------
# STEP 9: Save All DataFrames for Power BI
# -------------------------------
df_seg[final_cols].to_csv('segmented_customers.csv', index=False)
df_customers.to_csv('ready_customers.csv', index=False)
df_gender.to_csv('ready_gender.csv', index=False)
df_revenue.to_csv('ready_revenue.csv', index=False)
df_age_groups.to_csv('ready_age_groups.csv', index=False)
df_avg_spending.to_csv('ready_avg_spending.csv', index=False)
df_high_last_amount.to_csv('ready_high_last_amount.csv', index=False)

# -------------------------------
# STEP 10: Summary Stats (Optional)
# -------------------------------
print("\n🔹 Gender Distribution (%):")
print(df_seg['Gender'].value_counts(normalize=True) * 100)

print("\n🔹 Revenue by Category:")
print(df_revenue.sort_values(by='Revenue_Product', ascending=False))

print("\n🔹 Avg Spending Score by Age Group:")
print(df_seg.groupby('Age_Group')['Spending_Score'].mean())

print("\n✅ All steps complete. All cleaned files saved in the same folder as this notebook.")


✅ Final Table Preview:
   ID  Age Age_Group  Gender   Category  Income  Spending_Score  \
0   1   38     36–50  Female  Groceries   99342              90   
1   1   38     36–50  Female  Groceries   99342              90   
2   1   38     36–50  Female  Groceries   99342              90   
3   1   38     36–50  Female  Groceries   99342              90   
4   1   38     36–50  Female  Groceries   99342              90   

   Average_Income  Average_Spending_Score  Spend_vs_Peer  Income_vs_Peer  \
0         68764.0                    96.0           -6.0         30578.0   
1         46610.0                    92.0           -2.0         52732.0   
2        115734.0                    87.0            3.0        -16392.0   
3         70473.0                    81.5            8.5         28869.0   
4         70473.0                    81.5            8.5         28869.0   

                  Peer_Segment  Total_Customers  
0   Under-Spending High Earner              284  
1   Under-Spendin

In [3]:
import os
print(os.getcwd())

/Users/vodnalakarthik
